In [ ]:
%cd ./DNN-Prophecy
!pwd
!ls -lt dataset_models/airfoil_self_noise/

In [ ]:
!pip install -r requirements.txt

**LOAD DATA**

In [ ]:
import numpy as np
data = np.genfromtxt("./dataset_models/airfoil_self_noise/airfoil_self_noise.csv", delimiter=',')
print(data.shape)

inputstr = data[:,0:5]
outputstr = data[:,5]
#print(inputstr.shape)
#print(inputstr[0])

inputs = []
outputs = []
for indx in range(0, len(inputstr)):
  input = []
  for indx1 in range(0, len(inputstr[indx])):
    val = np.float32(inputstr[indx][indx1])
    input.append(val)
    #if (indx == 0):
    #  print(type(val), val)
    #  print(input)
  inputs.append(input)
  outputs.append(np.float32(outputstr[indx]))

print("EXAMPLE RAW INPUT AND OUTPUT.")
print(inputs[0])
print(outputs[0])

def z_score_normalize(data):
    """
    Applies z-score normalization (subtracts mean, divides by standard deviation) to the input data.

    Args:
        data (numpy.ndarray): The data to be normalized. Each row is a sample, and each column is a feature.

    Returns:
        normalized_data (numpy.ndarray): The normalized data.
        mean (numpy.ndarray): The mean of the original data (per feature).
        std (numpy.ndarray): The standard deviation of the original data (per feature).
    """
    # Convert data to NumPy array if it's a list
    data = np.asarray(data)

    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)

    # Avoid division by zero
    # Check if std is a scalar (for 1D data)
    if np.isscalar(std):
        if std == 0:
            std = 1
    else:  # For 2D data (or higher)
        std[std == 0] = 1

    normalized_data = (data - mean) / std
    return normalized_data, mean, std

(normalized_data, mean, std)= z_score_normalize(inputs)
print("EXAMPLE NORM INPUT AND OUTPUT.")
print(normalized_data[0])
(normalized_output, mean, std)= z_score_normalize(outputs)
print(normalized_output[0])


(1503, 6)
EXAMPLE RAW INPUT AND OUTPUT.
[800.0, 0.0, 0.3048, 71.3, 0.00266337]
126.201
EXAMPLE NORM INPUT AND OUTPUT.
[-0.6620218 -1.1463989  1.7992973  1.312917  -0.6448043]
0.1979379


**LOAD ONNX AND H5 MODELS**

In [ ]:
import torch
import onnx
import onnxruntime

onnx_model = onnx.load('./dataset_models/airfoil_self_noise/renamed_model.onnx')
onnx.checker.check_model(onnx_model)
from onnx import helper
graph = helper.printable_graph(onnx_model.graph)  # Get a printable representation of the graph
print(graph)  # Print the textual representation of the graph

In [ ]:
session = onnxruntime.InferenceSession("./dataset_models/airfoil_self_noise/renamed_model.onnx", None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
#print(input_name)
#print(output_name)


results = []
for indx in range(0,len(normalized_data)):
  input_data = (normalized_data[indx]).reshape(1, -1).astype(np.float32)
  result = session.run([output_name], {input_name: input_data})
  results.append(result[0][0][0])



from sklearn.metrics import r2_score
print(np.shape(results), np.shape(outputs))
r2 = r2_score(results,normalized_output)
print("R2 Score of ONNX MODEL:", r2)

In [ ]:
import numpy as np

import tensorflow as tf
import keras


print('Loading the model:')
model_h5=tf.keras.models.load_model("./dataset_models/airfoil_self_noise/model.h5")
print("Printing summary of the keras model:")
model_h5.summary()

# SPLIT TRAIN AND TEST SETS
len_train = int(0.80*len(normalized_data))

normalized_data_train = normalized_data[0:len_train]
normalized_data_test = normalized_data[len_train:]
normalized_output_train = normalized_output[0:len_train]
normalized_output_test = normalized_output[len_train:]

# GET PREDS
results_h5_train = model_h5.predict(normalized_data_train)
print(np.shape(results_h5_train), np.shape(normalized_output_train))
results_h5_train_flat = results_h5_train.flatten()
print(np.shape(results_h5_train_flat))
print(np.shape(normalized_output_train))
r2 = r2_score(results_h5_train_flat,normalized_output_train)
print("H5 R2 Score:", r2)

mean_abs_err = np.mean(np.abs(results_h5_train_flat - normalized_output_train))
print("H5 Mean Absolute Error:", mean_abs_err)

**LABEL REGRESSION OUTPUTS, 0 ( Y-YIDEAL > 0.192 (MAE)) ELSE 1**

In [ ]:
labels_train = []
op_vals_cor = []
op_vals_inc = []
for indx in range(0,len(normalized_data_train)):
  if (np.abs(normalized_output_train[indx] - results_h5_train_flat[indx]) >= 0.19200696):
    labels_train.append(0)
    op_vals_inc.append(results_h5_train_flat[indx])
  else:
    labels_train.append(1)
    op_vals_cor.append(results_h5_train_flat[indx])

print("TR COUNT 1:", labels_train.count(1))
print("TR COUNT 0:", labels_train.count(0))

print("MEAN CORRECT OP VAL:", np.mean(op_vals_cor))
print("MAX CORRECT OP VAL:", np.max(op_vals_cor))
print("MIN CORRECT OP VAL:", np.min(op_vals_cor))
std_dev_cor = np.std(op_vals_cor)
print("STD CORRECT OP VAL:", std_dev_cor)

print("MEAN INCORRECT OP VAL:", np.mean(op_vals_inc))
print("MAX INCORRECT OP VAL:", np.max(op_vals_inc))
print("MIN INCORRECT OP VAL:", np.min(op_vals_inc))
std_dev_inc = np.std(op_vals_inc)
print("STD CORRECT OP VAL:", std_dev_inc)



results_h5_test = model_h5.predict(normalized_data_test)
results_h5_test_flat = results_h5_test.flatten()


labels_test = []
for indx in range(0,len(normalized_data_test)):
  if (np.abs(normalized_output_test[indx] - results_h5_test_flat[indx]) >= 0.19200696):
    labels_test.append(0)
  else:
    labels_test.append(1)

print("TEST COUNT 1:", labels_test.count(1))
print("TEST COUNT 0:", labels_test.count(0))



**SENSITIVITY ANALYSIS**

In [ ]:
from maraboupy import Marabou
from maraboupy import MarabouNetworkONNX
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

## PICK AN INPUT FROM SET OF INPUTS WITH MAE < 0.192
cor_indices = np.where(np.array(labels_train) ==1)[0]
print(len(cor_indices))
print("INPUT:", normalized_data_train[cor_indices[0]])
print("IDEAL OUTPUT:", normalized_output_train[cor_indices[0]])
print("PREDICTION:", results_h5_train_flat[cor_indices[0]])

## VARY INPUT DIMENSIONS BY EPSILON
eps = 0.1
## CHECK OUTPUT IS WITHIN MAE
mae = 0.192


filename = "./dataset_models/airfoil_self_noise/renamed_model.onnx"
network_a = Marabou.read_onnx(filename)
print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()
input_flat = (normalized_data_train[cor_indices[0]]).flatten()
for indx in range(0,len(invars)):
  network_a.setLowerBound(invars[indx],input_flat[indx] - eps)
  network_a.setUpperBound(invars[indx],input_flat[indx] + eps)

print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
out_max = (results_h5_train_flat[cor_indices[0]]) + mae

network_a.setLowerBound(outvars[0],out_max) # F(X) > out_max should be UNSAT
#network_a.setUpperBound(outvars,out_min)

print("F(X) > out_max should be UNSAT")
options1 = Marabou.createOptions(verbosity = 1,timeoutInSeconds=200)
sat_unsat,vals,stats = network_a.solve(options = options1)
print("sat_unsat:", sat_unsat)

if (sat_unsat == 'sat'):
  print("SAT:")
  print("vals:", vals)


if (sat_unsat == 'unsat'):
   print("RULE PROVED!!")

#################################################
print("INPUT VARS")
network_a = Marabou.read_onnx(filename)
invars = network_a.inputVars[0][0].flatten()
input_flat = (normalized_data_train[cor_indices[0]]).flatten()
for indx in range(0,len(invars)):
  network_a.setLowerBound(invars[indx],input_flat[indx] - eps)
  network_a.setUpperBound(invars[indx],input_flat[indx] + eps)

print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
out_min = (results_h5_train_flat[cor_indices[0]]) - mae
network_a.setUpperBound(outvars[0],out_min)
print("F(X) < out_min should be UNSAT")
options1 = Marabou.createOptions(verbosity = 1,timeoutInSeconds=200)
sat_unsat,vals,stats = network_a.solve(options = options1)
print("sat_unsat:", sat_unsat)

if (sat_unsat == 'sat'):
  print("SAT:")
  print("vals:", vals)


if (sat_unsat == 'unsat'):
   print("RULE PROVED!!")





In [ ]:
np.save('./x_train_npy.npy',normalized_data_train)
np.save('./y_train_npy.npy',labels_train)
np.save('./x_test_npy.npy',normalized_data_test)
np.save('./y_test_npy.npy',labels_test)

In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -h

**Rules for behavior (0/1), from layer 5 based on (on/off) activations**

In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_5_output_0' -type 3 -acts True

In [ ]:
import pandas as pd
_output_path = "./results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_5_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_5_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_5_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_5_0_sup.csv",index=False)



****** RULES ********


**Rules from layer 4 based on neuron values**

In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_4_output_0' -type 3

In [ ]:
import pandas as pd
_output_path = "./results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_4_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_4_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_4_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_4_0_sup.csv",index=False)



****** RULES ********


In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_3_output_0' -type 3 -acts True

In [ ]:
import pandas as pd
_output_path = "./results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_3_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_3_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_3_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_3_0_sup.csv",index=False)



****** RULES ********


In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_2_output_0' -type 3

In [ ]:
import pandas as pd
_output_path = "./results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_2_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_2_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_2_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_2_0_sup.csv",index=False)



****** RULES ********


In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_1_output_0' -type 3 -acts True

In [ ]:
import pandas as pd
_output_path = "./results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_1_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_1_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_1_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_1_0_sup.csv",index=False)



****** RULES ********


In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_0_output_0' -type 3

In [ ]:
import pandas as pd
_output_path = "./results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_0_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_0_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_0_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_0_0_sup.csv",index=False)



****** RULES ********


In [ ]:

df_op = pd.read_csv("./rules_0_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 0 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_0_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 0 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_1_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 1 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_1_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 1 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_2_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 2 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_2_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 2 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_3_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 3 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_3_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 3 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_4_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 4 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_4_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 4 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_5_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 5 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_5_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 5 1:", train_f1,",",test_f1)



**PROVE RULES.**


In [ ]:
import shutil
marabou_path = '/usr/local/lib/python3.11/dist-packages/maraboupy/'
source_file1 = './dataset_models/MarabouNetworkONNX.py'
source_file2 = './dataset_models/ONNXParser.py'
    #destination_file1 = '/usr/local/lib/python3.11/dist-packages/maraboupy/MarabouNetworkONNX.py'
destination_file1 = marabou_path + '/MarabouNetworkONNX.py'
destination_file2 = marabou_path + '/parsers/ONNXParser.py'

shutil.copy(source_file1, destination_file1)
shutil.copy(source_file2, destination_file2)

'/usr/local/lib/python3.11/dist-packages/maraboupy//parsers/ONNXParser.py'

In [ ]:
#CREATE A RULES DIRECTORY UNDER ./results/airfoil_self_noise/
if (os.path.exists("./results/airfoil_self_noise/rules/") == False):
  os.makedirs("./results/airfoil_self_noise/rules")

!cp './rules_4_1_sup.csv' './results/airfoil_self_noise/rules/ruleset.csv'

In [ ]:
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/rules/' prove -h

**Post-Condition:**

**(min_val - MAE) <= F(X) <= (max_val + MAE)**

**where**

**min_val is the minimum value of the outputs of all inputs satisfying the rule.**

**max_val is the maximum value of the outputs of all inputs satisfying the rule.**


In [ ]:
# CREAT CONSTS FILE
import csv
consts = []
consts.append([0,'MIN',0.192])
#consts.append([0,'MAX',0.192])

with open('./results/airfoil_self_noise/rules/consts.csv', 'w') as f:
    writer = csv.writer(f)
    for indx in range(0,len(consts)):
      writer.writerow(consts[indx])

consts_read =[]
with open('./results/airfoil_self_noise/rules/consts.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        print(row)
        consts_read.append(row)

print("CONSTS READ:")
print(np.shape(consts_read))



**CHECK for counter-example to F(x) > MIN - MAE.**

**if there exists an x for which, F(x) < MIN-0.192.**

**Time-out: 20 mins**

In [ ]:
import time
start_time = time.time()
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/rules/' prove -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -mp '/usr/local/lib/python3.11/dist-packages/maraboupy/' -onx_map './dataset_models/airfoil_self_noise/airfoil_onnx_map.csv' -onx './dataset_models/airfoil_self_noise/renamed_model.onnx' -cp './results/airfoil_self_noise/rules/consts.csv' -label 1
elapsed_time1 =(time.time() - start_time)
print("ELAPSED TIME:", elapsed_time1)

**Post-Condition:**

**(min_val - MAE) <= F(X) <= (max_val + MAE)**

**where**

**min_val is the minimum value of the outputs of all inputs satisfying the rule.**

**max_val is the maximum value of the outputs of all inputs satisfying the rule.**


In [ ]:
# CREAT CONSTS FILE
import csv
consts = []
consts.append([0,'MAX',0.192])

with open('./results/airfoil_self_noise/rules/consts.csv', 'w') as f:
    writer = csv.writer(f)
    for indx in range(0,len(consts)):
      writer.writerow(consts[indx])

consts_read =[]
with open('./results/airfoil_self_noise/rules/consts.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        print(row)
        consts_read.append(row)

print("CONSTS READ:")
print(np.shape(consts_read))



**CHECK for counter-example to F(x) < MAX + MAE**

**if there exists an x for which, F(x) > MAX + 0.192**

**Time-out: 20 mins**

In [ ]:
import time
start_time = time.time()
!python -m prophecy.main -m './dataset_models/airfoil_self_noise/model.h5' -wd './results/airfoil_self_noise/rules/' prove -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -mp '/usr/local/lib/python3.11/dist-packages/maraboupy/' -onx_map './dataset_models/airfoil_self_noise/airfoil_onnx_map.csv' -onx './dataset_models/airfoil_self_noise/renamed_model.onnx' -cp './results/airfoil_self_noise/rules/consts.csv' -label 1
elapsed_time1 =(time.time() - start_time)
print("ELAPSED TIME:", elapsed_time1)